In [2]:
import os
import sys
from glob import glob

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import desispec
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra

from desiutil.log import get_logger, DEBUG

from desidiff.src.group_tiles import *
from desidiff.src.dates_to_process import *
from desidiff.src.coadd import *
from desidiff.src.scores import *
from desidiff.src.ContinuumFitFilter_desidiff import *

from timedomain.sp_utils import SkyPortal as sp
import requests
import datetime
import heapq
import time
import copy
import numpy
from astropy.time import Time
from astropy.table import Table, vstack, unique, SortedArray
import h5py

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from IPython import display
import warnings

In [3]:
#SkyPortal token:
secret_file = "/global/cfs/cdirs/desi/science/td/secrets/desidiff_sp.txt"
with open(secret_file, 'r') as file:
    token = file.read().replace('\n', '')
headers = {'Authorization': f'token {token}'}

filter_name = 'DESIDIFF'

warnings.filterwarnings('ignore')


In [31]:
# read in and store in one place all the fibermap information in the spectra files
#night_arr = getUnprocessedDates()
all_plots_pdf = PdfPages("All_plots.pdf")
count = 0
#for night in night_arr:   
night = 20210428
tile = 423
petal = 7 

for current_filename in glob(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/*/{night}/spectra-{petal}-{tile}*.fits"):
    #tile = current_filename.split('/')[-3]
    #petal = current_filename.split('-')[1]
    ### daily_spectra, the precursor to current spectra, before coadding, to select night, unique target ids, and individual target ids
    ### spectra.select functionality will not work once coadded with coaddition.coadd_cameras
    ### terrible hack to deal with one known case of fibermap['NIGHT' = 20210610] while night = 20210611
    try:
        daily_spectra = ((read_spectra(current_filename)).select(nights = night))
    except:
        continue

    table = Table.read(current_filename, format='fits',hdu=1, memmap=True) 
    ##### DAVE'S ADDITION ##############
    targetcols = [i for i in table.colnames if i[-7:] =='_TARGET']
    nonzerocheck = [True in k for k in [[j != 0 for j in table[targetcols][i]] for i in range(len(table))]]
    #a really ugly line, basically generates a list of bools, 
    #True if there is at least one nonzero element in all columns ending in _TARGET
    table.remove_rows([i for i, val in enumerate(nonzerocheck) if not val])
    #This gets the index of all False values from the previous list and removes those rows
    table = table['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC','FIBERSTATUS','NIGHT']
    ######## END DAVE'S ADDITION ############
    table = table[numpy.logical_and(table['OBJTYPE']=='TGT', table['FIBERSTATUS']==0)]
    ### Insert after last column but hard coded 8 == len(table.colnames)
    table.add_column(np.zeros(len(table)), name='FILTER_PASS', index=8)
    table['FILTER_PASS'] = table['FILTER_PASS'].astype(str)
    table.add_column(np.zeros(len(table)), name = 'perres_filter_broad', index = 9)
    table.add_column(np.zeros(len(table)), name = 'perres_filter_narrow', index = 10)
    
    for ref_filename in glob(f"/global/cfs/cdirs/desi/spectro/redux/*/tiles/cumulative/{tile}/*/coadd-{petal}-{tile}*.fits"):
        if ref_filename.split('/')[-6] == 'fuji' or ref_filename.split('/')[-6] == 'guadalupe':
            prev_spectra = read_spectra(ref_filename)
            ### creates a pdf whether or not any spectra on tile passes filters
            pdf_per_tile = PdfPages(str(tile)+".pdf")

            gen = (t for t in daily_spectra.target_ids()[numpy.where(numpy.logical_and(daily_spectra.fibermap['OBJTYPE']=='TGT', daily_spectra.fibermap['FIBERSTATUS']==0))] if t in prev_spectra.target_ids()[numpy.where(prev_spectra.fibermap['OBJTYPE']=='TGT')])
            for t in gen:
                count += 1
                print(count, t)
                #print(t)
                ### grab ra and dec values for use in SkyPortal functionality later
                ra, dec = table['TARGETID' == t]['TARGET_RA'], table['TARGETID' == t]['TARGET_DEC']
                ### match target ids and coadd_cameras
                current_spectra = desispec.coaddition.coadd_cameras(daily_spectra.select(targets = t))
                ref_spectra = desispec.coaddition.coadd_cameras(prev_spectra.select(targets = t))

                norm = normalization(current_spectra.flux, current_spectra.mask, ref_spectra.flux, ref_spectra.mask)

                # need to instantiate a Spectra object for the difference. 
                ### Using 'dif_spectra = Spectra()' is bugging on dif_spectra.mask type=NoneType, can't assign.
                #### dif_spectra = current_spectra
                ### copy.deepcopy() is deprecated as memory expensive
                dif_spectra = copy.deepcopy(current_spectra)
                #### any problem with hardcoding in 'brz' for key in the following:
                for key in (current_spectra.flux).keys():
                    current_spectra.flux[key] = current_spectra.flux[key]/norm
                    current_spectra.ivar[key] = current_spectra.ivar[key]*norm**2 
                    # subtraction of current and reference fluxes
                    dif_spectra.flux[key] = current_spectra.flux[key] - ref_spectra.flux[key]
                    ### couldn't dif_spectra.mask[key] == 2 by summing current.mask and reference.mask
                    # summation of current and reference masks
                    dif_spectra.mask[key] = current_spectra.mask[key] + ref_spectra.mask[key]
                    # inverted summation of current and spectra inverse variance
                    ### still returning RuntimeWarning: divide by zero encountered in true_divide but not in infinite loop for the moment
                    dif_spectra.ivar[key] = 1./(1./current_spectra.ivar[key] + 1./ref_spectra.ivar[key])

                # mean-subtracted difference
                dif_flux_clipped = clipmean(dif_spectra.flux, dif_spectra.ivar, dif_spectra.mask)

                # filters 
                # Difference spectrum may have broadband signal
                perband_filter = perband_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask)
                # fractional increase
                perband_inc = perband_increase(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ref_spectra.flux, ref_spectra.ivar, ref_spectra.mask)

                # Difference spectrum may have high-frequency signal
                perres_filter_broad = test_perconv_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ncon = 100, nsig = 17)
                perres_filter_narrow = test_perconv_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ncon = 5, nsig = 10)
                
                ### save filter values to table for statistical purposes
                table['TARGETID' == t]['perres_filter_broad'] = perres_filter_broad
                table['TARGETID' == t]['perres_filter_narrow'] = perres_filter_narrow
                print(table['TARGETID' == t]['perres_filter_broad'], table['TARGETID' == t]['perres_filter_narrow'])
                
                ### grab redrock data required for filters using redshift (zinfo)
                current_filename = (current_filename.replace('spectra','redrock')).replace('fits', 'h5')
                with h5py.File(current_filename, "r") as f:
                    zinfo = f['zfit'][str(t)]['zfit'][0]['z']

                # Search for signature lines of TDEs, only interested in Galaxies
                linetable = line_finder(dif_spectra.wave, dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, zinfo)
                Hline_score = Hline_filter(linetable)
                # deriv_score = deriv_filter(dif_flux_clipped, dif_spectra.ivar,dif_spectra.mask)
                #broadband
                bblogic = any(numpy.logical_and(numpy.array(list(perband_filter.values()))>10, numpy.array(list(perband_inc.values()))>0.25))
                narrowlinelogic = perres_filter_narrow >=2
                broadlinelogic = perres_filter_broad >=3
                # TDElogic = any([TDE_score == 2, TDE_score == 3, TDE_score == 4, TDE_score == 5])
                Hlinelogic = any([Hline_score >= 1])
                # derivlogic = any([deriv_score >= 3])
                logic = [bblogic, narrowlinelogic,  broadlinelogic, Hlinelogic]
                logic_name = ['Broadband', 'narrow line', 'broad line','Hline'] #must be in same order as logic!, use as mask
                logic_name = numpy.ma.masked_array(logic_name, mask = [not i for i in logic])
                plt.clf()

                plt.rcParams["figure.figsize"] = (20,6)
                if any(logic):
                    # add logic name to table
                    ####table['TARGETID' == t]['FILTER_PASS'] = str(logic)
                    # export to database of processed exposures
                    #processed(t, tile, night)
                    plt.figure()
                    for key in (current_spectra.flux).keys():
                        w=numpy.where(dif_spectra.mask[key][0] == 0)

                        plt.plot(dif_spectra.wave[key][w], dif_spectra.flux[key][0][w], color='red', label = 'Difference')
                        plt.plot(current_spectra.wave[key][w], current_spectra.flux[key][0][w], color='blue', alpha=0.5, label = 'New Spectrum')
                        plt.plot(ref_spectra.wave[key][w],ref_spectra.flux[key][0][w],color='green',alpha=0.5, label = 'Reference Spectrum')

                        plt.legend()
                        plt.xlabel('Wavelength (Å)')
                        plt.ylabel('Flux') 
                        plt.title(str(t) + "  " + str(night) + "  " + str(tile)  + "  " + str(logic_name))
                        #plt.show()
                        
                        plt.savefig(pdf_per_tile, format = 'pdf')
                        plt.savefig(all_plots_pdf, format = 'pdf')
                        plt.close()
                
pdf_per_tile.close()
print('pdf_per_tile closed')
all_plots_pdf.close()
print('all_plots_pdf closed')
               

INFO:spectra.py:291:read_spectra: iotime 0.400 sec to read spectra-7-423-thru20210428.fits at 2022-07-13T13:13:07.743427
INFO:spectra.py:291:read_spectra: iotime 0.294 sec to read coadd-7-423-thru20210428.fits at 2022-07-13T13:13:09.419946
1 39633456321269578
23.0 17.0
2 39633456321266068
11.0 12.0
3 39633456312880148
5.0 8.0
4 39633453867597961
8.0 6.0
5 39633456312879866
6.0 25.0
6 39633456321266429
10.0 45.0
7 39633456321266867
6.0 98.0
8 39633456317072111
16.0 24.0
9 39633453863406560
21.0 6.0
10 39633456321269619
4.0 5.0
11 39633456312879782
8.0 15.0
12 39633456312878681
6.0 24.0
13 39633456312880333
3.0 10.0
14 39633453867600495
5.0 20.0
15 39633456325460301
14.0 15.0
16 39633453867598241
20.0 6.0
17 39633456317071788
28.0 12.0
18 39633456321267332
13.0 8.0
19 39633456321267274
18.0 9.0
20 39633456317072806
30.0 25.0
21 39633456312880438
36.0 12.0
22 39633456317071963
13.0 14.0
23 39633456321268787
6.0 15.0
24 39633456329655170
13.0 3.0
25 39633458753962297
35.0 2.0
26 3963345632

KeyboardInterrupt: 

<Figure size 1440x432 with 0 Axes>

In [24]:
def test_perconv_SN(y, ivar, mask, ncon=5, nsig=10):
    newmask = dict(mask)
    newston = dict(y)
    newy = dict(y)
    conk = numpy.zeros(ncon) + 1.
    
    nTrue = 0
    for b in newston.keys():
        # mask lines that are smaller than the ncov

        f_logic = numpy.abs(y[b]*numpy.sqrt(ivar[b])) > 8
        index=0
        for _, g in itertools.groupby(f_logic):
            ng  = sum(1 for _ in g)
            for x in _:
                if x and ng<=3:
                    newy[b][index:index+ng] = 0
                index=index+ng

        newston[b]=numpy.array(numpy.convolve(newy[b][0], conk, mode='valid') / numpy.sqrt(numpy.convolve(1/ivar[b][0], conk, mode='valid')))
        newmask[b] = numpy.array(numpy.convolve(mask[b][0], conk, mode='valid'))
        
        w = numpy.where(newmask[b] == 0)[0]
        list_ston.append(newston[b][w])

        for gb0,_ in itertools.groupby(newston[b][w] > nsig):
            if gb0:
                nTrue=nTrue+1

    return nTrue

In [25]:
list_ston = []

In [33]:
print(list_ston)

[-1.37391082 -1.3997264  -1.40275645 ...  1.27691477  1.31034535
  1.30528184]
